In [1]:
import os
import numpy as np

import tensorflow as tf
from tensorflow.python.framework import ops

from load_data import *
from models.customlayers import *
from models.activations import *
from training import *
from download import *

import models.ConvAE as cae
L = tf.layers

import matplotlib.pyplot as plt
% matplotlib inline

ImportError: No module named 'moviepy'

# Model

Try first:

- input full video, output full video
- lstm encoder-decoder: at each timestep, input frame, output prediction for next frame
- start with fixed size: 20 frames (later work with true recurrent net)

Below is wrong; just instantiate lstm outside of function for now


In [2]:
def read_record(filepath_queue):
    reader = tf.RecordReader()
    _, serialized_example = reader.read(filepath_queue)
    
    features = tf.parse_single_example(
        serialized_example,
        features={
            'height': tf.FixedLenFeature([], tf.int64),
            'width': tf.FixedLenFeature([], tf.int64),
            'image': tf.FixedLenFeature([], tf.string)
        }
    )
    
    video = tf.decode_raw(features['image'], tf.uint8)   # feature may be renamed to video in future
    h = tf.cast(features['height'], tf.int32)
    w = tf.cast(features['width'], tf.int32)
    
    video_shape = tf.pack([-1, h, w, 3])
    video = tf.reshape(video, video_shape)
    
    return video
    
def inputs(split_type, batchsize, num_epochs): 
    if not num_epochs:
        num_epochs = None
        
    filepath = os.path.join(data_dir, '{}.tfrecords'.format(split_type))
    
    with tf.name_scope('input'):
        filepath_queue = tf.train.string_input_producer(
            [filepath], num_epochs=num_epochs
        )
    
    video = read_record(filepath_queue) # each video is a rank 4 tensor: [n_frames, x, y, c]
    
    videos = tf.train.shuffle_batch(
        [video],
        batchsize=batchsize, capacity=1024 + 2*batchsize, 
        num_threads=2, min_after_dequeue=128
    )
    
    return videos

In [3]:
def downsample(image):
    return L.max_pooling2d(image, 4, 4, name='downsampler')

def encoder(image):
    input_layer = tf.reshape(image, (-1, 240, 320, 3))
    
    downsampled = downsample(image)

    conv1 = L.conv2d(
        downsampled, name='conv1',
        filters=16, kernel_size=3, activation=selu
    )
        
    conv2 = L.conv2d(
        conv1, name='conv2',
        filters=32, kernel_size=3, activation=selu
    )
        
    conv3 = L.conv2d(
        conv2, name='conv3',
        filters=64, kernel_size=5, activation=selu
    )
        
    conv4 = L.conv2d(
        conv3, name='conv4',
        filters=128, kernel_size=5, activation=selu
    )
        
    shape = conv4.get_shape().as_list()
    print(shape)
    newdim = shape[1] * shape[2] * shape[3]
    print(newdim)
    conv4_flat = tf.reshape(conv4, (-1, newdim))
    print(conv4_flat.shape)

    dense1 = dense_reshape(
        conv4, name='dense1',
        units=2048, activation=selu
    )

    return dense1


def decoder(encoded):
    dense1 = L.dense(encoded, units=417792, activation=selu, name='dense1')

    dense1_reshaped = tf.reshape(dense1, (-1, 48, 68, 128))

    deconv1 = L.conv2d_transpose(
        dense1_reshaped, name='deconv1',
        filters=64, kernel_size=5, activation=selu    
    )

    deconv2 = L.conv2d_transpose(
        deconv1, filters=3, kernel_size=3, activation=selu,
        name='deconv2'
    )

    deconv3 = L.conv2d_transpose(
        deconv2, name='deconv3', 
        filters=32, kernel_size=3, activation=selu,
    )
    
    deconv4 = L.conv2d_transpose(
        deconv2, name='deconv4', 
        filters=3, kernel_size=3, activation=selu,
    )
    
    deconv4_reshaped = tf.transpose(deconv4, perm=(0, 3, 1, 2))

    return deconv4_reshaped

In [12]:
tf.reset_default_graph()

input_var = tf.placeholder(dtype=tf.float32, shape=(None, 3, 60, 80), name='input')
target_var = tf.placeholder(dtype=tf.float32, shape=(None, 3, 60, 80), name='target')

with tf.variable_scope('encoder'):
    encoded = encoder(input_var)
    
with tf.variable_scope('decoder'):
    decoded = decoder(encoded)

[None, 48, 68, 128]
417792
(?, 417792)


In [ ]:
def run_training(num_epochs, batchsize):
    videos = inputs('training', batchsize, num_epochs)
    
    with tf.variable_scope('encoder'):
        encoded = encoder(videos)
    
    with tf.variable_scope('lstm'):
        lstm_cell = tf.nn.rnn_cell.LSTMCell(num_units=1024)
        transitioned = tf.nn.dynamic_rnn(lstm_cell, encoded, )
    
    with tf.variable_scope('decoder'):
        decoded = decoder(transitioned)
        
    l2_weight = .01
    l2_term = tf.add_n([tf.nn.l2_loss(v) for v in tf.trainable_variables() if 'bias' not in v.name])

    loss = tf.reduce_mean(tf.pow(decoded - videos), 2)
    train_step = tf.train.AdamOptimizer().minimize(loss + l2_weight*l2_term)
    saver = tf.train.Saver()
    init = tf.global_variables_initializer()
    coord = tf.train.Coordinator()
    
    
    with tf.Session() as sesh:
        sesh.run(init)
        threads = tf.train.start_queue_runners(sess=sesh, coord=coord)
        try:
            step = 0
            while not coord.should_stop():
                _, loss_value = sesh.run([train_op, loss])
                
                if step % 500 == 0:
                    print('Step {} loss: {:.4f}'.format(step, loss_value))
                    
        except tf.errors.OutOfRangeError:
            print('Done; loss: {:.4f}'.format(step, loss_value))
            
        finally:
            coord.request_stop()
            
        coord.join(threads)

In [ ]:
tf.reset_default_graph()

input_var = tf.placeholder(dtype=tf.float32, shape=(None, 3, 60, 80), name='input')
target_var = tf.placeholder(dtype=tf.float32, shape=(None, 3, 60, 80), name='target')

with tf.variable_scope('encoder'):
    encoded = cae.encoder(input_var)
    
with tf.variable_scope('decoder', reuse=True):
    decoded = cae.decoder(encoded)

loss = tf.reduce_mean(tf.pow(decoded - target_var, 2))
train_step = tf.train.AdamOptimizer().minimize(loss)
saver = tf.train.Saver()
init = tf.global_variables_initializer()